In [ ]:
import os
import pandas as pd
from pypdf import PdfReader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from crewai import Agent, Task, Crew

# 1. 환경 설정
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

# 2. PDF 기반 RAG 설정 (RFP 내용 추출)
def get_rfp_context(pdf_path):
    print(f"[{pdf_path}] PDF 문서 분석 중...")
    reader = PdfReader(pdf_path)
    raw_text = ""
    # 다이아몬드 기판 등 주요 과제가 포함된 초반 페이지 집중 추출 
    for i in range(min(10, len(reader.pages))):
        text = reader.pages[i].extract_text()
        if text: raw_text += text
    
    splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
    docs = splitter.create_documents([raw_text])
    vectorstore = FAISS.from_documents(docs, OpenAIEmbeddings())
    
    # 한국어 핵심 키워드로 컨텍스트 검색
    relevant_docs = vectorstore.similarity_search("연구개발 목표, 정량적 성능지표, 기술적 요구사항, 관리지침", k=7)
    return "\n".join([doc.page_content for doc in relevant_docs])

# 3. 에이전트 정의 (사용자 지정 페르소나 적용)
extractor = Agent(
    role="공공 및 민간 입찰 요구사항 분석 전문가",
    goal="RFP PDF 문서에서 기술적, 관리적 요구사항을 정확히 추출하여 구조화된 체크리스트 테이블을 생성한다.",
    backstory="""당신은 20년 경력의 조달 전문 컨설턴트입니다. 수만 페이지의 RFP를 분석해온 경험을 바탕으로, 
    문장 사이에 숨겨진 발주처의 의도와 필수 요구사항을 찾아내는 데 탁월한 능력을 갖추고 있습니다. 
    당신이 작성한 체크리스트는 제안서의 성패를 결정하는 기준점이 됩니다.""",
    allow_delegation=False,
    verbose=True
)

matcher = Agent(
    role="제안서 컴플라이언스(Compliance) 감수 전문가",
    goal="제안요구사항 체크리스트와 작성된 제안서를 대조하여 항목별 일치 여부를 판정하고 미흡한 부분을 지적한다.",
    backstory="""당신은 아주 까다롭기로 유명한 제안 평가 위원입니다. 
    제안서 내에 RFP가 요구한 키워드와 솔루션이 적절히 포함되었는지 현미경처럼 들여다봅니다. 
    단순히 단어의 포함 여부를 넘어, 요구사항이 실질적으로 구현 가능한 형태로 기술되었는지 판단하여 일치 여부 테이블을 작성합니다.""",
    allow_delegation=False,
    verbose=True
)

reviewer = Agent(
    role="전략 제안 품질 및 리스크 검토 전문가",
    goal="최종 제안서의 표절 여부를 확인하고, 논리적으로 취약한 3가지 지점과 그에 대한 대응 논리를 생성한다.",
    backstory="""당신은 경쟁사의 입장에서 우리 제안서를 비판적으로 분석하는 Red Team 리더입니다. 
    심사위원이 질의응답(Q&A) 시간에 던질법한 날카로운 질문을 미리 예측하며, 
    전체 문맥의 유사도를 검토하여 독창성을 확보하고 제안서의 설득력을 극대화하는 역할을 수행합니다.""",
    allow_delegation=True,
    verbose=True
)

# 4. 데이터 로드 및 태스크 정의
rfp_content = get_rfp_context("example_rfp_배관관리.pdf")

# 테스트용 제안서 초안 (작성 중인 텍스트)
proposal_draft = """
[제안서] AI 기반 지능형 배관 관리 및 예방정비 솔루션
1. 귀사의 배관 관리, '터지고 나서' 수리하시겠습니까?
대부분의 기업이 배관 내부가 막히거나 녹물이 나온 후에야 막대한 비용을 들여 교체 공사를 진행합니다. 
하지만 사후 대응 방식은 업무 중단, 자산 가치 하락, 그리고 예방 정비 대비 5배 이상의 지출을 초래합니다.
**[A기업]**은 생성형 AI와 정밀 진단 기술을 결합하여, 보이지 않는 배관 속 리스크를 데이터로 관리하고 비용을 혁신적으로 절감해 드립니다.
2. 주요 서비스 내용 (AI 기반 3단계 케어)
① AI 정밀 진단 및 리스크 예측
디지털 내시경 분석: AI가 배관 내부 영상을 분석하여 부식 정도, 슬러지 퇴적률을 수치화합니다.
AI 예측 리포트: 건축물 연식과 사용량을 분석하여 향후 3년간 발생할 수 있는 누수 및 폐쇄 위험도를 예측합니다.
② 맞춤형 지능형 세척 (Smart Flushing)
고압 펄스 공법: 배관에 무리를 주지 않으면서 스케일만 정밀하게 제거하는 최첨단 공법을 적용합니다.
최적 주기 설정: AI가 귀사의 설비 상태에 가장 경제적인 세척 주기를 산출하여 과잉 정비를 방지합니다.
③ 생성형 AI 관리 대시보드
자동 분석 보고서: 복잡한 수치 대신, AI가 생성한 '쉬운 요약 보고서'를 통해 의사결정권자에게 관리 성과를 즉시 보고할 수 있습니다.
24/7 모니터링: 수질 센서와 연동하여 이상 징후 발생 시 즉각 알림을 발송합니다.
3. 도입 시 기대 효과 (Economic Value)
구분도입 전 (사후 수리)도입 후 (AI 예방 정비)
비용 절감노후 배관 전면 교체 (고비용)정기 세척으로 수명 2배 연장 (60% 절감)
업무 연속성누수 시 공장/사무실 폐쇄비가동 시간 최소화 (야간/주말 작업)
관리 편의성문제 발생 시 업체 수동 수수AI 자동 스케줄링 및 리포트 제공
자산 가치설비 노후화로 가치 하락깨끗한 수질 및 관리 이력으로 가치 보존
4. [A기업]만의 차별점
기술력: 업계 최초 생성형 AI 기반 배관 노후도 판독 알고리즘 보유
신뢰성: 공공기관 및 대단지 산업단지 레퍼런스 확보
사후보장: 서비스 후 문제 발생 시 즉각 출동 및 무상 AS 보증 제도 운영
5. 향후 추진 일정
현장 무료 진단: AI 스캐닝 장비를 통한 배관 상태 샘플링 (D-Day)
분석 리포트 제출: 진단 결과 및 예상 절감 비용 제안 (D+3)
연간 케어 계약: 귀사 맞춤형 정기 관리 서비스 개시 (D+7)
"보이지 않는 곳의 관리가 기업의 진정한 경쟁력입니다."
본 제안과 관련하여 상세한 상담이나 무료 현장 진단을 원하시면 언제든 연락 주시기 바랍니다.
[A기업] 사업본부 배상연락처: 02-xxx-xxxx이메일: contact@acorp.ai홈페이지: www.acorp.ai
"""

task1 = Task(
    description=(
        f"입력된 RFP PDF 파일의 전수 조사를 통해 발주처가 요구하는 모든 기술적, 관리적, 사업적 요구사항을 추출하세요. "
        f"각 요구사항은 ID를 부여하고, 카테고리(기능, 보안, 유지보수 등), 세부 내용, 중요도를 포함해야 합니다. "
        f"표(Table) 형태로 정리하여 후속 에이전트가 참조할 수 있도록 하세요. 한국어로 답변하세요.\n\n"
        f"[RFP 추출 내용]\n{rfp_content}"
    ),
    expected_output="요구사항 ID, 카테고리, 요구사항 명칭, 상세 내용, 중요도가 포함된 xlsx 테이블 형식의 리스트",
    agent=extractor,
    output_file="1_요구사항_체크리스트.md"
)

task2 = Task(
    description=(
        f"extractor가 생성한 요구사항 테이블과 사용자가 작성한 제안서 텍스트를 대조하세요. "
        f"각 요구사항 ID별로 제안서에 내용이 포함되었는지 확인하고, '일치/미흡/누락' 상태를 판정하세요. "
        f"내용이 포함되었다면 제안서의 어느 부분에 해당 내용이 있는지 위치를 명시해야 합니다. 한국어로 답변하세요.\n\n"
        f"[제안서 초안]\n{proposal_draft}"
    ),
    expected_output="요구사항 ID, 일치 여부(O/△/X), 제안서 내 관련 섹션, 검토 의견이 포함된 이행 분석 테이블",
    agent=matcher,
    context=[task1],
    output_file="2_일치_여부_보고서.md"
)

task3 = Task(
    description=(
        "사용자의 최종 제안서를 분석하여 독창성을 확인하고, 심사위원의 관점에서 논리적으로 허술하거나 "
        "공격받기 쉬운 '취약한 부분' 3가지를 도출하세요. "
        "단순한 지적에 그치지 않고, 실제 발표(PT) 현장에서 답변할 수 있는 설득력 있는 대응 시나리오를 작성하세요. 한국어로 답변하세요."
    ),
    expected_output="3가지 핵심 취약점 및 각 취약점에 대한 논리적 답변(Q&A)이 포함된 전략 리포트",
    agent=reviewer,
    context=[task2],
    output_file="3_최종_전략_리포트.txt"
)

# 5. 크루 실행 및 결과 저장 함수 (CSV 변환 로직 포함)
def export_md_to_csv_korean(md_file, csv_file):
    try:
        # 다양한 인코딩으로 읽기 시도
        for enc in ['utf-8', 'cp949', 'euc-kr']:
            try:
                with open(md_file, 'r', encoding=enc) as f:
                    lines = [l.strip() for l in f.readlines() if "|" in l]
                if lines: break
            except UnicodeDecodeError: continue
            
        if len(lines) > 2:
            data = [[cell.strip() for cell in row.split("|") if cell.strip()] for row in lines]
            df = pd.DataFrame(data[2:], columns=data[0])
            # utf-8-sig: 엑셀에서 한글이 깨지지 않게 하는 인코딩 
            df.to_csv(csv_file, index=False, encoding='utf-8-sig')
            print(f"✅ 파일 생성 완료: {csv_file}")
    except Exception as e:
        print(f"❌ 파일 변환 중 오류 발생: {e}")

# 실행 및 저장
proposal_crew = Crew(
    agents=[extractor, matcher, reviewer],
    tasks=[task1, task2, task3],
    verbose=True
)

print("\n### 통합 분석 시스템을 가동합니다 ###")
result = proposal_crew.kickoff()

# 각 단계별 결과물을 CSV 및 텍스트로 저장
export_md_to_csv_korean("1_요구사항_체크리스트.md", "1_요구사항_체크리스트.csv")
export_md_to_csv_korean("2_일치_여부_보고서.md", "2_일치_여부_보고서.csv")

print("\n" + "="*50)
print("분석 완료! 다음 파일들이 생성되었습니다.")
print("1. 1_요구사항_체크리스트.csv")
print("2. 2_일치_여부_보고서.csv")
print("3. 3_최종_전략_리포트.txt")
print("="*50)

[example_rfp_배관관리.pdf] PDF 문서 분석 중...


2026-01-15 13:19:51,229 - 11980 - __init__.py-__init__:537 - WARNING: Overriding of current TracerProvider is not allowed



### 통합 분석 시스템을 가동합니다 ###
 [DEBUG]: == Working Agent: 공공 및 민간 입찰 요구사항 분석 전문가
 [INFO]: == Starting Task: 입력된 RFP PDF 파일의 전수 조사를 통해 발주처가 요구하는 모든 기술적, 관리적, 사업적 요구사항을 추출하세요. 각 요구사항은 ID를 부여하고, 카테고리(기능, 보안, 유지보수 등), 세부 내용, 중요도를 포함해야 합니다. 표(Table) 형태로 정리하여 후속 에이전트가 참조할 수 있도록 하세요. 한국어로 답변하세요.

[RFP 추출 내용]
정기 관리의  경제적  이득을  생성형  AI를 통해 시각화하여  설득 논리 개발. 
③ 비대면  자동화  영업 채널 구축 
• AI 챗봇 상담: 건물 정보 입력 시 예상 세척 비용 및 관리 필요성을  즉시 
답변하는  생성형  AI 챗봇 도입. • 성과 대시보드 : 고객사가  다수의  사업장을  관리할  경우, 전체 사업장의  
배관 상태를  한눈에  볼 수 있는 통합 관리 대시보드  제공. 
 
3. 핵심 요구사항  (Technical & Business 
Requirements)  
구분 요구사항  상세 
기술적  측면 - 생성형  AI(LLM) 기반의  맞춤형  컨설팅  리포트  생성 기능 
 
 
- 배관 오염도  이미지  인식을  위한 Computer Vision 모델 연동 
 
 
- 기존 ERP/CRM 시스템과의  데이터  호환성  
비즈니스  
측면 - B2B 신규 거래처  확보를  위한 영업 자동화  툴 제공 
 
 
- 연간 계약 체결을  유도하는  '리스크  관리' 관점의  마케팅  
메시지  설계 
 
 
- 진입 장벽 해소를  위한 '무료 진단 이벤트 ' 등 프로모션  전략 
유지보수  - 실시간  수질 데이터  모니터링  및 이상 징후 발생 시 긴급 출동 
연동 기능 
 
4. 기대 효과 
• 매출 안정성  확보: 단발성  공사가  아닌 연간 정기 구독 계약을  통한 예측 
가능한  수익 구조 확립. • 영업